# Initial class creation

In [3]:
class MyTradingStrategy:

    def __init__(self, name):

        self.__name = name

    def generate_signal(self, price_data):
        print('This method is intended to be over ridded')
        return 'Hold'
    
    @property
    def name(self):
        return self.__name
    


In [4]:
MyBaseObj = MyTradingStrategy('Awesome Strategy')
MyBaseObj.generate_signal(13)
MyBaseObj.name

This method is intended to be over ridded


'Awesome Strategy'

# SMA trading strategy

In [7]:
class MySMATradingStrategy(MyTradingStrategy):

    def __init__(self, swindow, lwindow):
        self.__swindow = swindow
        self.__lwindow = lwindow
        super().__init__('MY SMA Trading strategy')

    def generate_signal(self, price_data):

        if len(price_data[-self.__lwindow:]) < self.__lwindow:
            return 'Hold'
        
        short_avg = sum(price_data[-self.__swindow:]) / self.__swindow
        long_avg = sum(price_data[-self.__lwindow:]) / self.__lwindow

        if short_avg > long_avg:
            return 'Buy'
        elif short_avg < long_avg:
            return 'Sell'
        else:
            return 'Hold'

    @property
    def swindow(self):
        return self.__swindow
    
    @property
    def lwindow(self):
        return self.__lwindow

In [9]:
ObjStrategy = MySMATradingStrategy(3, 5)
print(ObjStrategy.generate_signal([12, 14, 15, 14, 2, 3, 8, 9, 7, 14]))
print(ObjStrategy.lwindow)
print(ObjStrategy.swindow)
print(ObjStrategy.name)

Buy
5
3
MY SMA Trading strategy
